In [1]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
#you are running the lab in your  browser, so we will install the libraries using ``piplite``
import piplite
await piplite.install(['pandas'])
await piplite.install(['numpy'])

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

In [9]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [10]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [11]:
await download(path, "Weather_Data.csv")
filename ="Weather_Data.csv"

In [12]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [13]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])


In [14]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)


In [15]:
df_sydney_processed.drop('Date',axis=1,inplace=True)


In [16]:
df_sydney_processed = df_sydney_processed.astype(float)


In [17]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size = 0.2, random_state = 10);
print (x_train.shape, x_test.shape, y_train.shape)

(2616, 66) (655, 66) (2616,)


In [19]:
LinearReg = LinearRegression().fit(x_train,y_train)

In [20]:
predictions = LinearReg.predict(x_test)

In [22]:
LinearRegression_MAE = np.mean(np.abs(predictions - y_test)) 
LinearRegression_MSE = np.mean((predictions - y_test)**2) 
LinearRegression_R2 = LinearReg.score(x_test, y_test)

In [23]:
Report = pd.DataFrame({'MAE': LinearRegression_MAE, 'MSE': LinearRegression_MSE, 'R2': LinearRegression_R2}, index = [0])
Report


,MAE,MSE,R2
0,0.256319,0.115721,0.42713


In [24]:
features = preprocessing.StandardScaler().fit(features).transform(features.astype(float));
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size = 0.2, random_state = 10);
KNN = KNeighborsClassifier(n_neighbors = 4).fit(x_train,y_train)

In [25]:
predictions = KNN.predict(x_test);

In [26]:
KNN_Accuracy_Score = accuracy_score(y_test,predictions)
KNN_JaccardIndex = jaccard_score(y_test,predictions)
KNN_F1_Score = f1_score(y_test,predictions)

In [27]:
Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 10).fit(x_train,y_train);

In [28]:
predictions = Tree.predict(x_test)


In [29]:
Tree_Accuracy_Score = accuracy_score(y_test,predictions)
Tree_JaccardIndex = jaccard_score(y_test,predictions)
Tree_F1_Score = f1_score(y_test,predictions)
print (Tree_Accuracy_Score, Tree_JaccardIndex, Tree_F1_Score)

0.7938931297709924 0.4327731092436975 0.6041055718475073


In [30]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size = 0.2, random_state = 1);

In [31]:
LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train,y_train);

In [32]:
predictions = LR.predict(x_test);
predictions_prob = LR.predict_proba(x_test);

In [33]:
LR_Accuracy_Score = accuracy_score(y_test,predictions)
LR_JaccardIndex = jaccard_score(y_test,predictions)
LR_F1_Score = f1_score(y_test,predictions)
LR_Log_Loss = log_loss(y_test,predictions_prob)
print(LR_Accuracy_Score,LR_JaccardIndex,LR_F1_Score,LR_Log_Loss)

0.8259541984732824 0.5021834061135371 0.6686046511627907 0.3886031385809171


In [34]:
SVM = svm.SVC(kernel='rbf').fit(x_train, y_train);

In [35]:
predictions = SVM.predict(x_test);

In [36]:
SVM_Accuracy_Score = accuracy_score(y_test,predictions)
SVM_JaccardIndex = jaccard_score(y_test,predictions)
SVM_F1_Score = f1_score(y_test,predictions)
print(SVM_Accuracy_Score,SVM_JaccardIndex,SVM_F1_Score)

0.8290076335877863 0.4666666666666667 0.6363636363636364


In [37]:
Report = pd.DataFrame(columns=['Model Name', 'Accuracy', 'Jaccard','F1-Score','LogLoss']);
new_rows = [{'Model Name': 'KNN' , 'Accuracy': KNN_Accuracy_Score, 'Jaccard': KNN_JaccardIndex,'F1-Score': KNN_F1_Score,'LogLoss': None},
            {'Model Name': 'Decision Tree' , 'Accuracy': Tree_Accuracy_Score, 'Jaccard': Tree_JaccardIndex,'F1-Score': Tree_F1_Score,'LogLoss': None},
           {'Model Name': 'Logistic Regression', 'Accuracy': LR_Accuracy_Score, 'Jaccard': LR_JaccardIndex,'F1-Score': LR_F1_Score,'LogLoss': LR_Log_Loss},
           {'Model Name': 'SVM', 'Accuracy': SVM_Accuracy_Score, 'Jaccard': SVM_JaccardIndex,'F1-Score': SVM_F1_Score,'LogLoss': None}]
for row in new_rows:
    Report = Report.append(row, ignore_index=True)
print(Report)

            Model Name  Accuracy   Jaccard  F1-Score   LogLoss
0                  KNN  0.760305  0.241546  0.389105       NaN
1        Decision Tree  0.793893  0.432773  0.604106       NaN
2  Logistic Regression  0.825954  0.502183  0.668605  0.388603
3                  SVM  0.829008  0.466667  0.636364      None
